In [22]:
import requests
from bs4 import BeautifulSoup
import datetime

In [10]:
class Icing():
    url = 'https://freemeteo.cz/pocasi/mlada-boleslav/hodinova-predpoved/dnes/?gid=3070544&language=czech&country=czech-republic'
    def __init__(self):
        self.temp = None
        self.wind = None
        self.hum = None
        self.dew = None
        self.icing = False

    def get_forecast(self):
        soup = BeautifulSoup(requests.get(Icing.url).content, "html.parser")
        table = soup.find("div", class_ = "today").find('tbody')
        results = dict()
        for indx1, row in enumerate(table.find_all('tr')):
            if indx1 in [2,4,5]:
                cell = row.find_all('td')[1]
                if indx1 == 2:
                    self.temp = int(cell.text.replace('°C',''))
                elif indx1 == 4:
                    self.wind = int(cell.find(text=True, recursive=False).replace(' Km/h',''))
                else:
                    self.hum = int(cell.text.replace('%',''))

    def calculate_dew(self):
        self.dew = self.temp - ((100 - self.hum) / 5)

    def check_icing(self):
        if self.dew < 0 and self.wind > 15:
            self.icing = True

In [11]:
icing = Icing()
icing.get_forecast()
icing.calculate_dew()
icing.check_icing()
icing.icing

False

In [28]:
class IcingApi():
    key = '7786e1769e4b18a984bbe2e0cfe41427'
    lat = '50.4114'
    long = '14.9032'
    exclude = 'current,minutely,daily,alerts'
    url = f'https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={long}&exclude={exclude}&appid={key}&units=metric'

    def __init__(self):
        self.results = self._get_forecast()
    
    def _get_forecast(self):
        r = requests.get(IcingApi.url)
        return r.json()

    def predict_icing(self):
        prediction = []
        for hour in self.results['hourly']:
            icing = True if hour['dew_point'] < 0 and hour['wind_speed'] < 15 else False
            prediction.append(
                {'time': datetime.datetime.fromtimestamp(hour['dt']).isoformat(),
                'icing': icing})
        return prediction

In [29]:
icing = IcingApi()
icing.predict_icing()

[{'time': '2022-10-12T19:00:00', 'icing': False},
 {'time': '2022-10-12T20:00:00', 'icing': False},
 {'time': '2022-10-12T21:00:00', 'icing': False},
 {'time': '2022-10-12T22:00:00', 'icing': False},
 {'time': '2022-10-12T23:00:00', 'icing': False},
 {'time': '2022-10-13T00:00:00', 'icing': False},
 {'time': '2022-10-13T01:00:00', 'icing': False},
 {'time': '2022-10-13T02:00:00', 'icing': False},
 {'time': '2022-10-13T03:00:00', 'icing': False},
 {'time': '2022-10-13T04:00:00', 'icing': False},
 {'time': '2022-10-13T05:00:00', 'icing': False},
 {'time': '2022-10-13T06:00:00', 'icing': False},
 {'time': '2022-10-13T07:00:00', 'icing': False},
 {'time': '2022-10-13T08:00:00', 'icing': False},
 {'time': '2022-10-13T09:00:00', 'icing': False},
 {'time': '2022-10-13T10:00:00', 'icing': False},
 {'time': '2022-10-13T11:00:00', 'icing': False},
 {'time': '2022-10-13T12:00:00', 'icing': False},
 {'time': '2022-10-13T13:00:00', 'icing': False},
 {'time': '2022-10-13T14:00:00', 'icing': False},
